## Predict_with_DAM

In [1]:
!pip install net

ERROR: Could not find a version that satisfies the requirement net
ERROR: No matching distribution found for net


In [3]:
import pandas as pd
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences

In [4]:
# Configs

MAX_LEN = 128

# 如果ConnectionError就把local_files_only改成True
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',local_files_only=False,do_lower_case=True)

In [5]:
context = "呃来电是,感谢你对联通的支持嘛!从下个月开始,每月25元就可以领取两款VIP会员来用了。20多个会员随机选择，呃，联通还是返还你4个月话费，每月10元返还的话费是可以直接抵扣你月租费用的。田先生你好!你下个月收短信可以领取来用了好吧。\t嗯，好的，好。\t嗯，那好，那耽误20秒，感谢你的支持啊！每月25元的钻石会员，7月生效。推动点联通还会返还您10个月话费，每月10元话费会在3月10号前到账，你到时候根据短信提醒你在微信公众号或音乐领取就可以就可以用了，好吧。\t嗯好，谢谢！"
df_r = pd.read_excel("./data/嘉音讯_重庆联通_未来会员STT入库表.xlsx")

re = df_r["文案"].unique()

df = pd.DataFrame({"context":context,"response":re})

In [6]:
df.head(8)

,context,response
0,"呃来电是,感谢你对联通的支持嘛!从下个月开始,每月25元就可以领取两款VIP会员来用了。20...",您好，这边是联通客服来电的，请问手机尾号****是你在用的对吧？
1,"呃来电是,感谢你对联通的支持嘛!从下个月开始,每月25元就可以领取两款VIP会员来用了。20...",来电是感谢您对联通的支持。\n您从下个月开始，每月只要25元就可以领取两个VIP会员。\n因...
2,"呃来电是,感谢你对联通的支持嘛!从下个月开始,每月25元就可以领取两款VIP会员来用了。20...",就每个月25元，这边有话费40元返还给您抵扣月租的，分四月每月到账10元，希望可以帮您几个月...
3,"呃来电是,感谢你对联通的支持嘛!从下个月开始,每月25元就可以领取两款VIP会员来用了。20...",不会影响您的套餐的，每月25元，联通还会返还40元话费给您，分四月每月到账10元，希望可以帮...
4,"呃来电是,感谢你对联通的支持嘛!从下个月开始,每月25元就可以领取两款VIP会员来用了。20...",没关系啊，这边也是可以给您续的，而且我们这个是有二十多种会员，您每个月可以任意选两个用，每个...
5,"呃来电是,感谢你对联通的支持嘛!从下个月开始,每月25元就可以领取两款VIP会员来用了。20...",就耽误您一两分钟时间，现在有会员给到您用，联通还会返还40元话费给您，分四月每月到账10元，...
6,"呃来电是,感谢你对联通的支持嘛!从下个月开始,每月25元就可以领取两款VIP会员来用了。20...",好的，感谢您的支持，跟你说一下会员怎么领取，您根据短信在微信公众号“沃音乐”领取就好了。\n...
7,"呃来电是,感谢你对联通的支持嘛!从下个月开始,每月25元就可以领取两款VIP会员来用了。20...",好的，使用中您可以下载“中国联通”APP查询您的消费情况、关注最新活动福利，祝您生活愉快，再见！


### Embedding

In [7]:
def vectorize(text, MAX_LEN=MAX_LEN, tokenizer=tokenizer):

    input_ids = []
    for t in text:
        # so basically encode tokenizing , mapping sentences to thier token ids after adding special tokens.
        encoded_sent = tokenizer.encode(
            t,  # Sentence which are encoding.
            add_special_tokens=True,  # Adding special tokens '[CLS]' and '[SEP]'
        )
        input_ids.append(encoded_sent)

    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN , truncating="pre", padding="post")

    attention_masks = []
    for sent in input_ids:
        # Generating attention mask for sentences.
        #   - when there is 0 present as token id we are going to set mask as 0.
        #   - we are going to set mask 1 for all non-zero positive input id.
        att_mask = [int(token_id > 0) for token_id in sent]

        attention_masks.append(att_mask)

    return attention_masks, input_ids

In [8]:
_, X_c = vectorize(df.context)
_, X_r = vectorize(df.response)

In [9]:
from six import b
data = {}
data[b('c')] = X_c
data[b('r')] = X_r
data[b('y')] = [1,0,0,0,0,0,0,0]

In [10]:
import pickle

with open('./data/predict.pkl','wb') as f:
    pickle.dump(data,f)

### Predict

In [10]:
!python -m pip install paddlepaddle

### Select Response

In [12]:
file_path = r'C:\Users\g2018\Desktop\DAM\models-release-1.5\PaddleNLP\dialogue_model_toolkit\deep_attention_matching\saved_models\pscore.txt'
data = []
with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()
        tokens = line.split("\t")

        if len(tokens) != 2:
            continue

        data.append((float(tokens[0]), int(tokens[1])))

In [13]:
pre_r = []
for i in range(0,len(data)):
  pre_r.append((data[i][0],df.response[i]))

In [14]:
sort_data = sorted(pre_r, key=lambda x: x[0], reverse=True)

In [15]:
context

'呃来电是,感谢你对联通的支持嘛!从下个月开始,每月25元就可以领取两款VIP会员来用了。20多个会员随机选择，呃，联通还是返还你4个月话费，每月10元返还的话费是可以直接抵扣你月租费用的。田先生你好!你下个月收短信可以领取来用了好吧。\t嗯，好的，好。\t嗯，那好，那耽误20秒，感谢你的支持啊！每月25元的钻石会员，7月生效。推动点联通还会返还您10个月话费，每月10元话费会在3月10号前到账，你到时候根据短信提醒你在微信公众号或音乐领取就可以就可以用了，好吧。\t嗯好，谢谢！'

In [16]:
sort_data[0][1]

'好的，使用中您可以下载“中国联通”APP查询您的消费情况、关注最新活动福利，祝您生活愉快，再见！'